# Learning to Rank for IR
[original PDF](http://wwwconference.org/www2009/pdf/T7A-LEARNING%20TO%20RANK%20TUTORIAL.pdf)

## This tutorial
+ IRのための順序学習
  - そのほかの分野でのランキング問題ではない
+ Supervised learning
  - unsupervised, semi-supervised学習ではない
+ ベクトル空間で学ぶ
  - グラフやそのほかのデータ構造を扱うものではない
+ 主にSIGIR, WWW, ICML, NIPSへの論文に向けたもの

## Introduction
「情報の洪水に飲み込まれていないかい?」

### Webについての真実
+ `www.worldwidewebsize.com`によれば, 250億以上のページがウェブ上に存在する.
+ 主要な検索エンジンは何百億ものWebページをインデックス化している
+ CUIL.com では1200億のウェブページをインデックスづけしている

「情報は検索無くして成り立たない」

### Ranking is Essential
インデックスづけされたドキュメントのレポジトリに対してクエリを投げつける. そのクエリに基づいてランキングモデルが関連度を順序づけてくれる.

### Application of Ranking
+ document retrieval
+ collaborative filtering
+ Key term extraction
+ Definition finding
+ Important email routing
+ Sentimant analysis
+ Product rating
+ Anti Web spam
+ etc...

いろいろな場面で`ランキング`は利用されている.

### Senarios of Ranking
ドキュメント検索の具体例
+ クエリに対する関連度に基づいて単純にドキュメントをランクづけする
+ ドキュメント間の類似性, web構造, 多様性を順序づけの過程で考慮する(相対的ランキング)
+ 複数の候補者のランクづけされたリストを集計してより優れた順序リストを生成する
+ Webページの属性がどれほどランキング結果に影響を与えるのかを考える

### Evaluation of Ranking Results
+ ランダムに抽出された多数のクエリから構成されるテストセットを作成し, `relevance judgment`を行う
+ テストセットに含まれる特定のクエリに対するランキング結果を`evaluation measure`に基づいて評価する
+ テストセット全体への平均的な精度を全体のランキングパフォーマンスとして扱う.

### Collecting Documents for A Query
+ `TREC`で利用される`Pooling Strategy`
  - 関連性があるかもしれないドキュメントを`participating system`から取得する

### Relevance Judgment
+ Degree of relevance (関連度)
  - Binary: relevant vs. irrelevant (関係あり vs. 無関係)
  - Multiple ordered categories: Perfect > Excellent > Good > Fair > Bad (段階あり)
+ Pairwise preference (1組みで比較)
  - "ドキュメントAはドキュメントBよりもより関連性がある"
+ Total order (全順序)
  - 関連度に応じてそのランク{A, B, C, ...}が付与される
  
### Evaluation Measure
関連性の判断についての評価. Judgmentがどれほどよかったかについて.

#### MAP
+ クエリ$q$について$k$番目の精度
    + $P@k = \frac{上位k件までで関連ありDOCS数}{k}$
+ クエリ$q$の`AP (average precision)`は以下のように定義される.
$$
AP = \frac{\sum_{k} P@k \cdot l_{k}}{関連ありDOCS数}
$$

例えば, 上位5件が関連(アリ, ナシ, アリ, ナシ, アリ)であった場合...
$$
AP = \frac{1}{3} \cdot (\frac{1}{1} + \frac{2}{3} + \frac{3}{5})
$$
で計算でき, その値は0.76くらいとなる.

ポイントは, 上位$k$件のうち, 関連のあったものだけに対して$P@k$が計算され, 
$AP$はそれらの和を関連のあるドキュメント数で割ったものとして求められることである.

$MAP$は全てのクエリに対しての$AP$の平均値として計算される.

In [18]:
def precision_at_k(li, k):
    """
    li: @{list of bool} ランキングリスト
    k: @{int} 上位k件目
    """
    if li[k-1]:
        return sum(li[:k]) / k
    return 0

def average_precision(li):
    return sum([precision_at_k(li, i+1) for i in range(len(li))]) / sum(li)

eg_list = [True, False, True, False, True]
eg_list_2 = [True, False, False, True, True]
print(average_precision(eg_list))
print(average_precision(eg_list_2))

0.7555555555555555
0.7000000000000001


#### NDCG
クエリ$q$に対するランキングの$k$件目までの$DCG$は以下のように計算される. 
ただし, $rel_{i}$は$i$番目の提案の上位である妥当性を表す.
$$
DCG_{k} = \sum_{i=1}^{k} \frac{2^{rel_{i}} - 1}{log_{2}(i+1)}
$$

また, $NDCG$は次のように計算される.
$$
NDCG_{k} = \frac{DCG_{k}}{idealDCG_{k}}
$$

例えば, 上位3件のそれぞれのスコアが3, 0, 2である場合は, 
$$
DCG_{3} = \frac{2^{3} - 1}{log_{2}2} + \frac{2^{0} - 1}{log_{2}3} + \frac{2^{2} - 1}{log_{2}4}\\
DCG_{3} = 5
$$

となる.

### Evaluation Measure のまとめ
+ Position-based: 順位が明示的に利用されている
  - より上位のオブジェクトの方が重要
  - 関連性の順序 vs. 関連度を表すスコア
  - スコアは連続的ではなく, 微分も不可能である場合が多い

## Conventional Ranking Models
昔昔のはなし. どういうランクモデルがあったのだろうか. その特徴をまとめる.
+ Query-dependent
  - Boolean model, 拡張版Boolean model
  - Vector space model, 潜在意味的インデックス化(LSI)など
  - BM25モデル, 統計的言語モデルなど
  - Spanベースモデル, 距離集約モデルなど
+ Query-independent
  - PageRank, TrustRank, BrowseRankなど
  
このように, クエリに依存する順位づけモデルと依存しない順位づけモデルが存在することがわかる.

### 従来モデルの問題
+ マニュアル化されたパラメータチューニングが大抵の場合において難しい. 
  + 特にパラメータの数が多い場合など.
+ パラメータチューニングが過学習を引き起こしやすい

### Machine Learningを組みこもう
+ 機械学習は効果的なツール
  + パラメータを自動的にチューニングしてくれる
  + いろいろな要因を組み込める
  + 正則化などの手法を利用することで過学習を防ぐことができる

ここで現れるのが... "Learning to rank"!!!

一般に, 機械学習を利用したランク学習は`learning to rank`と呼ばれる.

## Learning to rank
最近の`Learning to rank`に関わる手法から
1. 特徴ベース
2. 差別的なトレーニング
といった性質がある.

### 特徴ベース(Feature based)
+ ドキュメントは特徴ベクトルとして表現される
+ たくさんの特徴量を組み込んで処理を行うことが可能に！

### 差別的トレーニング(descriminative training)
+ トレーニングデータに基づいた自動的な処理が可能
  - `Input space`, `output space`, `hypothesis space`, `loss function`
  - 確率論的な説明があまり必要ない
+ 現実の検索エンジンで求められている性質
  - 数多くのユーザフィードバックやログが存在する
  
### Learning to rank Framework
* * *
![フレームワーク](fig01.png)
* * *


### Key questions to answer
+ `learning to rank`アルゴリズムは互いにどのように類似し, どのように異なるのか.
+ それぞれの強みと弱みはなんだ?
+ 調査を行いたいランキングに対するユニークな理論的な問題はなんだ?
+ 経験から, どのアルゴリズムがうまくうごくのか?
+ 将来的な問題点はなんだ?

## Learning to Rank Algorithm
+ pointwise アプローチ
+ pairwise アプローチ
+ listwise アプローチ

### Pointwise approach
***
![pointwise](fig02.png)
***

#### 具体例
+ 以下のランキング生成に応用
    + 回帰を利用した`subset ranking`
    + 分類
        + Discriminative model for IR
        + MCRank
    + Ordinal regression (順序回帰?)
        + `Pranking`
        + 最大化マージンランキング

##### Subset Ranking using Regression
(D. Cossock and T. Zhang, COLT 2006)

関連度(relevance degree)を実数値としてみなし, ランキング関数を回帰を利用して学習する. Loss functionを以下のように設定し, 最小化を測る.
$$
L(f;x_{j}, y_{j}) = |f(x_{j}) - y_{j}|^{2}
$$

##### Discriminative model for IR
ランク学習の段階で分類が利用されている. 関連性のあるドキュメントを正のサンプル(positive example)として利用し, 関連性のないドキュメントを負のサンプル(negative sample)として扱う. ($y_{j} = +1, y_{j} = -1$)
$L(f; x_{j}, y_{j})$は0/1損失関数$I_{\{f(x_{j})\neq y_{j}\}}$

SVMのhinge関数を利用した最適化アルゴリズムも存在する.

##### McRank
多クラス分類をランク学習に応用.
$$
\hat{y_{j,k}} = P(y_{j} = k), f(x_{j}) = \sum_{k=0}^{K-1}\hat{p_{j,k}}k
$$

#### Pointwiseの問題点
+ 関連度がクエリに依存しすぎる場合が存在
+ 損失関数においてドキュメントの順位が見えない. ゆえに関連度の薄いものに重要度を割きすぎる場合がある

### Pairwise Approach
***
![pairwise](fig03.png)
***

Preference function $h(x_{u}, x_{v}) = 2I_{f(x_{u}) \gt f(x_{v})} - 1$. 
関数Iは添え字の式が真であれば1, 偽であれば0の値をとる.

#### 具体例
+ Learning to order things
+ RankNet and Frank
+ RankBoost
+ RankingSVM
+ Multi-hyperplane ranker
+ IR-SVM

